In [5]:
import pandas as pd
import csv
from google_play_scraper import reviews, Sort
from datetime import datetime
import time # Tambahkan untuk jeda, jika perlu scraping dalam jumlah sangat besar

# --- KONFIGURASI ---
# GANTI DENGAN APP ID APLIKASI YANG ANDA INGINKAN
APP_ID = 'com.bpjstku' # Aplikasi BPJSTKU
# Tentukan jumlah minimal komentar yang ingin diambil
MIN_KOMENTAR = 500

def get_comments_playstore(app_id: str, count: int) -> pd.DataFrame:
    """
    Mengambil komentar dari Google Play Store menggunakan google-play-scraper
    dan mengembalikannya dalam bentuk DataFrame.
    """

    print(f"Memulai scraping {count} komentar dari App ID: {app_id}...")

    try:
        # Mengambil semua data hingga batas 'count'
        result, _ = reviews(
            app_id,
            lang='id',
            country='id',
            sort=Sort.NEWEST,
            count=count,
            filter_score_with=None
        )
    except Exception as e:
        print("---------------------------------------------------------------------------------")
        print(f"🛑 Gagal mengambil data dari Google Play Store untuk ID: {app_id}")
        print(f"🛑 Pastikan APP ID benar dan library 'google-play-scraper' sudah terinstal.")
        print(f"Error: {e}")
        print("---------------------------------------------------------------------------------")
        return pd.DataFrame() # Mengembalikan DataFrame kosong jika gagal

    # Konversi langsung ke DataFrame untuk penanganan data yang lebih mudah
    df = pd.DataFrame(result)

    if df.empty:
        print("Tidak ada komentar yang ditemukan.")
        return pd.DataFrame()

    # --- Feature Engineering & Pembersihan Data Awal ---

    # 1. Pilih dan Ganti Nama Kolom yang Relevan
    df_clean = df[['userName', 'content', 'at', 'score']].copy()
    df_clean.rename(columns={
        'userName': 'author',
        'content': 'comment',
        'at': 'publishedAt'
    }, inplace=True)

    # 2. Pembersihan Format Tanggal (publishedAt)
    # Memastikan kolom 'publishedAt' bertipe string dengan format yang konsisten.
    # Objek datetime dari scraper sudah cukup baik, kita hanya perlu format string.

    # Menangani kasus jika ada nilai non-datetime (meskipun jarang terjadi)
    df_clean['publishedAt'] = df_clean['publishedAt'].apply(
        lambda x: x.strftime("%Y-%m-%d %H:%M:%S")
        if isinstance(x, datetime) else 'N/A'
    )

    print(f"Scraping selesai. Jumlah data yang berhasil diambil: {len(df_clean)}")
    return df_clean

def save_to_csv(df: pd.DataFrame, filename: str):
    """
    Menyimpan DataFrame ke file CSV.
    """
    if df.empty:
        print(f"Tidak ada data untuk disimpan ke {filename}.")
        return

    try:
        # Menggunakan metode pandas to_csv() yang lebih efisien
        df.to_csv(filename, index=False, encoding='utf-8', quoting=csv.QUOTE_ALL)

        print(f"✅ Berhasil menyimpan {len(df)} komentar ke '{filename}'")
    except Exception as e:
        print(f"🛑 Gagal menyimpan ke CSV: {e}")


if __name__ == '__main__':
    # Pastikan Anda telah menginstal library yang diperlukan:
    # !pip install google-play-scraper pandas

    df_comments = get_comments_playstore(APP_ID, MIN_KOMENTAR)

    # Ganti nama file output
    output_filename = f'komentar_playstore_{APP_ID.replace(".", "_")}.csv'

    save_to_csv(df_comments, output_filename)

Memulai scraping 500 komentar dari App ID: com.bpjstku...
Scraping selesai. Jumlah data yang berhasil diambil: 500
✅ Berhasil menyimpan 500 komentar ke 'komentar_playstore_com_bpjstku.csv'


In [4]:
pip install google-play-scraper